<a href="https://colab.research.google.com/github/young-hwanlee/gnn_examples/blob/main/3_gnn_link_level_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/young-hwanlee/gnn_examples.git

Cloning into 'gnn_examples'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 104 (delta 59), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (104/104), 3.01 MiB | 9.66 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
# ## To avoid some errors (출처: https://life-is-potatoo.tistory.com/95 [삶은 감자:티스토리]) ==========
# ## 1st method
# import torch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# def format_pytorch_version(version):
#     return version.split('+')[0]

# TORCH_version = torch.__version__
# TORCH = format_pytorch_version(TORCH_version)
# def format_cuda_version(version):
#     return 'cu' + version.replace('.', '')

# CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

## Ensure that the PyTorch and the PyG are the same version
# !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
# !pip install torch-geometric

## 2nd method
import os
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# !pip install torch_geometric

## Ensure that the PyTorch and the PyG are the same version
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install pytorch_metric_learning

## Both methods are too slow.

2.4.0+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
## Import required packages ====================================================================
# import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GraphSAGE
from torch_geometric.nn import InnerProductDecoder
from torch_geometric.loader import LinkNeighborLoader

import numpy as np
import random
from sklearn.model_selection import train_test_split

In [4]:
## Set up the seed for reproducibility ============================================================
seed = 42

torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)     # if use multi-GPU

np.random.seed(seed)

random.seed(seed)

In [5]:
# Load a dataset =================================================================================
from torch_geometric.datasets import Reddit

dataset = Reddit(root="/tmp/Reddit")

In [6]:
print(f'Dataset: {dataset}:')
print('============================================================================================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0].to(device)
print(data)       # Get the 1st graph object
print(data.x[0, :])

Dataset: Reddit():
Number of graphs: 1
Number of features: 602
Number of classes: 41
Data(x=[232965, 602], edge_index=[2, 114615892], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965])
tensor([ 1.2334e+00,  9.0430e+00, -9.2328e-01,  1.0542e+00, -1.1125e+00,
        -2.0630e-02,  4.2540e-02,  2.1520e+00, -9.0830e-01,  7.1384e-01,
         1.1512e+00, -1.2357e+00,  1.1699e+00, -1.2657e+00, -8.6435e-01,
         3.5655e-01, -1.0509e+00,  1.3829e-01, -2.8985e-01, -4.5366e-01,
        -1.7766e-01,  7.8746e-01, -1.2298e+00, -9.3718e-01, -2.8484e-02,
        -6.2839e-01, -1.5315e-01,  4.0115e-02,  8.6607e-01, -3.5893e-01,
        -9.7886e-02,  1.0300e+00, -1.5091e-01, -1.1921e+00,  7.9493e-01,
         3.7253e-01,  1.7400e-01, -2.5684e-01, -1.2256e+00, -3.7878e-01,
        -1.3944e+00,  4.4133e-01,  8.8975e-01,  7.1985e-02,  1.5196e+00,
         1.2643e+00,  3.3659e-01, -8.5132e-01,  1.6793e+00, -5.1868e-01,
         1.9735e+00,  1.2463e+00,  4.9956e-01, -7.1537e-01, -3.5

In [7]:
# ## Visualize the GNN networks ===================================================================
# ## Import packages required for visualization
# import matplotlib.pyplot as plt
# import networkx as nx
# from torch_geometric.utils import to_networkx
# from sklearn.manifold import TSNE

# %matplotlib inline

# def visualize_graph(G, color, ax=None):
#     if ax is None:
#         ax = plt.gca()
#     ax.set_xticks([])
#     ax.set_yticks([])

#     # plt.sca(ax)
#     nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
#                      node_color=color, cmap="Set2")

# # plt.cla()
# # plt.clf()

# ## plt.subplot -> stated-based
# ## plt.subplots -> objected-oriented

# ## 1st method
# # fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 5))

# ## 2nd method
# plt.figure(figsize=(5, 5))
# G = to_networkx(data, to_undirected=True)
# visualize_graph(G, color=data.y)

# plt.show()

# ## Unable to plot due to lack of RAM

In [8]:
# ! pip install pyg-lib -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html         # Not working
! pip install pyg-lib -f https://data.pyg.org/whl/torch-${TORCH}.html

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html


In [9]:
n_train_data = sum(data.train_mask)
n_val_data = sum(data.val_mask)
n_test_data = sum(data.test_mask)

print(f'Number of train data : {n_train_data}')
print(f'Number of validation data : {n_val_data}')
print(f'Number of test data : {n_test_data}')
print(f'Number of data : {n_train_data + n_val_data + n_test_data}')

Number of train data : 153431
Number of validation data : 23831
Number of test data : 55703
Number of data : 232965


In [10]:
# Create a mini-batch loader =====================================================================
loader = LinkNeighborLoader(data, num_neighbors=[25, 10], batch_size=128)

In [11]:
# Create your GNN model ========================================================================
class GNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(seed)
        # Choose between different GNN building blocks:
        self.encoder = GraphSAGE(data.num_features, 16, num_layers=2)
        self.decoder = InnerProductDecoder()

    def forward(self, x, edge_index, edge_label_index):
        x = self.encoder(x, edge_index)
        return self.decoder(x, edge_label_index)

num_classes = data.y.max().item() + 1
model = GNN().to(device)
print(model)

GNN(
  (encoder): GraphSAGE(602, 16, num_layers=2)
  (decoder): InnerProductDecoder()
)


In [12]:
# Train your GNN model =========================================================================
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train(model, train_loader, mask, n_epochs):
    model = model.to(device=device)
    best_val_loss = float('inf')
    best_model = None

    train_loss_hist, train_acc_hist = [], []
    val_loss_hist, val_acc_hist = [], []

    model.train()
    for epoch in range(n_epochs + 1):
        train_loss, train_acc = 0, 0

        for data in train_loader:
            data = data.to(device)

            optimizer.zero_grad()       # Clear gradients
            out = model(data)
            loss = criterion(out, data.y)     # Compute the loss solely based on the training nodes
            train_loss += loss.item() / len(train_loader.dataset)
            train_acc += torch.sum(out.argmax(dim=1) == data.y) / len(train_loader.dataset)
            loss.backward()     # Derive gradients
            optimizer.step()     # Update parameters based on gradients

        if epoch % 20 == 0:
            train_loss_hist.append(train_loss)
            train_acc_hist.append(train_acc)

            val_loss, val_acc = test(model, val_loader)
            val_loss_hist.append(val_loss)
            val_acc_hist.append(val_acc)
            print(f'Epoch : {epoch:03d}, | Train loss : {train_loss:.4f}, | Train accuracy : {train_acc:.4f} | Valid loss : {val_loss:.4f}, | Valid accuracy : {val_acc:.4f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model

    return best_model, train_loss_hist, train_acc_hist, val_loss_hist, val_acc_hist

def test(model, test_loader, mask):
    loss, correct = 0, 0

    model.eval()
    for data in loader:
        data = data.to(device)
        out = model(data.x[mask], data.edge_index, data.edge_label_index)
        loss += criterion(out[mask], data.y[mask]).item() / len(loader)
        correct += torch.sum(out.argmax(dim=1)) == data.y[mask]

    acc = correct / len(loader.dataset)

    return acc